<a href="https://colab.research.google.com/github/fabriciocgf/fcc_machine_learning/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.7.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-01-31 12:36:37--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  1.59MB/s    in 0.2s    

2022-01-31 12:36:39 (1.59 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-01-31 12:36:39--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [3]:
train_df = pd.read_csv(train_file_path, sep='\t', names=['type','text'], header=None)
one_hot_encoded_train = pd.get_dummies(train_df, columns = ['type'])
one_hot_encoded_train.tail()

,text,type_ham,type_spam
4174,just woke up. yeesh its late. but i didn't fal...,1,0
4175,what do u reckon as need 2 arrange transport i...,1,0
4176,free entry into our £250 weekly competition ju...,0,1
4177,-pls stop bootydelious (32/f) is inviting you ...,0,1
4178,tell my bad character which u dnt lik in me. ...,1,0


In [4]:
test_df = pd.read_csv(test_file_path, sep='\t', names=['type','text'], header=None)
one_hot_encoded_test = pd.get_dummies(test_df, columns = ['type'])
one_hot_encoded_test.tail()

,text,type_ham,type_spam
1387,true dear..i sat to pray evening and felt so.s...,1,0
1388,"what will we do in the shower, baby?",1,0
1389,where are you ? what are you doing ? are yuou ...,1,0
1390,ur cash-balance is currently 500 pounds - to m...,0,1
1391,not heard from u4 a while. call 4 rude chat pr...,0,1


In [5]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')

#Aplicando uma função para tokenizar por palavra
one_hot_encoded_train['text'] = one_hot_encoded_train.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
one_hot_encoded_test['text'] = one_hot_encoded_test.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [19]:
import re

from nltk.corpus import stopwords
nltk.download('stopwords')
language = 'portuguese'

#Criando a lista de stopwords
stopwords = stopwords.words(language)
stopwords = list(set(stopwords))

def remove_stopwords(words):
    """Remover as Stopwords das palavras tokenizadas"""
    new_words = []
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words

def to_lowercase(words):
    new_words = []
    for word in words:
        new_word = str(word).lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

one_hot_encoded_train['text'] = one_hot_encoded_train.apply(lambda row: normalize(row['text']), axis=1)
one_hot_encoded_test['text'] = one_hot_encoded_test.apply(lambda row: normalize(row['text']), axis=1)

vocab = one_hot_encoded_train['text'].values.tolist()
vocab = [item for sublist in vocab for item in sublist]
vocab = list(set(vocab))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Creating a mapping from unique characters to indices
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)

def catch(func, *args, handle=lambda e : 0, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return handle(e)

def text_to_int(text):
  return np.array([catch(lambda : word2idx[c]) for c in normalize(text)])

text_as_int = text_to_int(['what', 'will', 'we', 'do', 'in', 'the', 'shower', 'baby'])
print(text_as_int)

[4940 2389  676 3794 5403 2722 4758]


In [8]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ' '.join(idx2word[ints])

print(int_to_text(text_as_int))

what will we in the shower baby


In [9]:
one_hot_encoded_train['text'] = one_hot_encoded_train.apply(lambda row: text_to_int(row['text']), axis=1)
one_hot_encoded_test['text'] = one_hot_encoded_test.apply(lambda row: text_to_int(row['text']), axis=1)
train_labels = one_hot_encoded_train.pop('type_ham').to_numpy()
test_labels = one_hot_encoded_test.pop('type_ham').to_numpy()
oone_hot_encoded_train = one_hot_encoded_train.drop('type_spam', axis=1)
one_hot_encoded_test = one_hot_encoded_test.drop('type_spam', axis=1)

In [10]:
print(len(train_labels))

4179


In [11]:
one_hot_encoded_test.tail()

,text
1387,"[1018, 0, 4505, 7200, 2897, 151, 5329, 1084, 0..."
1388,"[4940, 2389, 676, 3794, 5403, 2722, 4758]"
1389,"[7073, 3793, 4041, 4940, 3793, 4041, 6171, 379..."
1390,"[1930, 2038, 3803, 325, 7256, 1083, 7200, 65, ..."
1391,"[4335, 3464, 6272, 7124, 2814, 6859, 538, 2957..."


In [12]:
from keras.preprocessing import sequence
VOCAB_SIZE = len(vocab)

MAXLEN = 100

train_data = sequence.pad_sequences(one_hot_encoded_train['text'].to_numpy(), MAXLEN)
test_data = sequence.pad_sequences(one_hot_encoded_test['text'].to_numpy(), MAXLEN)

In [13]:
argu = 1389
print(train_data[argu],train_labels[argu])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0  328 5092 1694 5403
  381 4259] 1


In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          250464    
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 260,961
Trainable params: 260,961
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=5, validation_split=0.2)

Epoch 1/5
105/105 [==============================] - 10s 71ms/step - loss: 0.2305 - acc: 0.9219 - val_loss: 0.1091 - val_acc: 0.9809
Epoch 2/5
105/105 [==============================] - 9s 81ms/step - loss: 0.0527 - acc: 0.9838 - val_loss: 0.0497 - val_acc: 0.9868
Epoch 3/5
105/105 [==============================] - 8s 78ms/step - loss: 0.0319 - acc: 0.9898 - val_loss: 0.0885 - val_acc: 0.9773
Epoch 4/5
105/105 [==============================] - 6s 54ms/step - loss: 0.0218 - acc: 0.9934 - val_loss: 0.0659 - val_acc: 0.9797
Epoch 5/5
105/105 [==============================] - 6s 58ms/step - loss: 0.0124 - acc: 0.9970 - val_loss: 0.0764 - val_acc: 0.9773


In [16]:
results = model.evaluate(test_data, test_labels)
print(results)

44/44 [==============================] - 0s 11ms/step - loss: 0.0613 - acc: 0.9842
[0.06133221089839935, 0.9841954112052917]


In [17]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  tokens = nltk.word_tokenize(pred_text)
  text_int = text_to_int(tokens)
  text_int = sequence.pad_sequences([text_int], MAXLEN)[0]
  pred = np.zeros((1,MAXLEN))
  pred[0] = text_int
  result = model.predict(pred)
  prediction =  result[0][0]
  if prediction > 0.5:
    out = 'ham'
  else:
    out = 'spam'

  return [prediction, out]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.9999538, 'ham']


In [18]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

You passed the challenge. Great job!
